# OceanHackWeek2024 - Project Presentation

## Project - Xarray accessor "load-by-step"

### Project description:

**A Xarray accessor to download large quantities of data automatically splitting a large request in smaller requests.**

Note: A Xarray accessor is simillar to a child class, e.g.:

```
MyClass(xr.DataArray):
    def my_method(self, arg1, arg2):
        ...
```

But it is the recommended way of doing thins with Xarray as show [here](https://docs.xarray.dev/en/stable/internals/extending-xarray.html)

### Project Members:

* Marcelo Andrioni
* João Pedro Amorin

### Which problem are we trying to solve?

**TL;DR: Downloading large amounts of data from remote servers (e.g.: THREDDS, HYRAX, etc.) using Xarray without server timeouts or silent failures and saving it in a "reasonable" (not thousands!!!) number of files for further analysis.**

The Xarray module is extensively used in the geosciences for its ability to handle multi-dimensional data, provide metadata support, and offer a wide range of analysis and visualization capabilities. Its intuitive interface and compatibility with other Python libraries make it a valuable tool for geoscientific research and data analysis. It's basically Pandas for N-dimensional data.

By default, Xarray tries to do everything in "lazy" mode until the data has to be acctually loaded.

In [10]:
import xarray as xr

ds = xr.tutorial.open_dataset("air_temperature_gradient")
da = ds["Tair"]
print(da._in_memory)

False


If you perform an operation that really needs the data, like multiplication, saving to disk, etc, then everything is actually loaded in memory.

In [15]:
foo = da * 2
print(da._in_memory)

True


This is not a problem if you are working with local Datasets/DataArrays or small (few MB order of magnitude) remote Datasets/DataArrays.
But if you are accessing data from a remote server like [THREDDS](https://www.unidata.ucar.edu/software/tds/), [Hyrax](https://www.opendap.org/software/hyrax-data-server/), [ERDDAP](https://github.com/ERDDAP/erddap), etc, large requests (> 100MB order of magnitude) can fail.

These kind of servers are the "default" mode of serving model hindcast/forecast and satellite data on the interet. Some examples:

* [HYCOM](https://tds.hycom.org/thredds/catalog.html)
* [NCEI](https://www.ncei.noaa.gov/thredds/catalog.html)
* [NOMADS](https://nomads.ncep.noaa.gov/)

Due to the multitude of possible configuration options for these servers, the requests can even fail "silentlty", that is, without raising an error.
When this happens, the download seems to finish without problems, but in fact the data is full of NaN, zeros, or garbage values like 1e39.

Thin can severely impact any subsequent workflows that depend on the data.

An example would be trying to download current forecast data to run an oil simulation, getting data filled with zeros, and using that data in the oil drift model without realizing it...

It's a wild example, but it actually happened to me. Since the oil drift model also used the wind forecast as a forcing, the oil particles didn't just stay in place. Only later did I realize that the currents were not used in the simulation and everything had to be redone. It was not a real oil spill, but it could have been.

One way of solving this is to split a large request in smaller requests, e.g.:

In [ ]:
import numpy as np
import pandas as pd

dts = pd.date_range("2000-01-01", "2024-12-31", freq="1D")
url = "http://foobar"

for dt in dts[0:-1]:
    with xr.open_dataset(url).sel(time=slice(dt, dt - np.timedelta64(1,'s'))):
        ds.to_netcdf(f"foobar_{%Y%m%d:dt}.nc")

But with this approach, at the end the user will have 9132 files that will have to "joined" (with NCO, CDO, xr.open_mfdataset, etc) to perform the desired analysis.
It can be done, but this adds a lot of gruntwork and introduces additional points of failure in the whole process. For someone that "just wants to do an analysis", having to learn all these different methods and libraries can consume a precious ammount of time.

### Proposed solution

**TL;DR: a Xarray accessor that splitts a large requests in smaller requests internally and returns a single Dataset/DataArray to the user, all in a single line of code.**

During OHW24 we developed a python module called "Xarray load-by-step". The main code can be seen [here](https://github.com/oceanhackweek/ohw24_proj_xarray_load_by_step_us/blob/main/src/load_by_step/_load_by_step.py).

If anybody wants to follow allong, the module can be installed on your "conda env" with the following command:

`pip install load_by_step@git+https://github.com/oceanhackweek/ohw24_proj_xarray_load_by_step_us`

You start by importing xarray and the new module

In [16]:
import xarray as xr
import load_by_step

If you check the new module, there is almost nothing there that the user can call directly. Basically just an option in set_option to disable TQDM and don't show the progress bar.

In [17]:
dir(load_by_step)

['DALoadByStep',
 'DSLoadByStep',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_load_by_step',
 '_options',
 'get_options',
 'set_options']

To acctually use the new module you need to look for the "lbs" module in the Dataset/DataArray instance scope.

In [25]:
ds = xr.tutorial.open_dataset("air_temperature_gradient")
da = ds["Tair"]
print(da.lbs)

Now you are able to call the loading methods. For a DataArray the available methods are `load_by_step` and `load_by_bytesize`.

**Note**: For this first demonstration we are going to use a small local file. Later on, we are going to shown some "real life" examples downloading data from remote servers. 

In [32]:
da2 = da.lbs.load_by_step(time=5)
print(da._in_memory, da2._in_memory)

Loading '13.2KB' of 'Tair' between time=[2014-12-30T18:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|█████████████████████████████| 584/584 [00:03<00:00, 167.03it/s]


False True


The command above is going to split the loading process of the DataArray in blocks of 5 elements along the time dimension. The progress bar shows the size (in KB, MB, etc) of the block and the start and ending elements.

You can even split along two or more dimensions.

In [33]:
da2 = da.lbs.load_by_step(time=100, lon=30)
print(da._in_memory, da2._in_memory)

Loading '23.0KB' of 'Tair' between time=[2014-12-27T00:00:00.000000000, 2014-12-31T18:00:00.000000000], lon=[275.0, 330.0]: 100%|███████████| 60/60 [00:00<00:00, 123.53it/s]


False True


You can see some more arguments in the docstring

In [44]:
da.lbs.load_by_step?

Signature:
da.lbs.load_by_step(
    *,
    indexers: Optional[Mapping[str, Annotated[int, Gt(gt=0)]]] = None,
    seconds_between_requests: typing.Annotated[float, Ge(ge=0)] = 0,
    **indexers_kwargs: Optional[Annotated[int, Gt(gt=0)]],
) -> xarray.core.dataarray.DataArray
Docstring:
Load the DataArray in memory splitting the loading process along one
or more dimensions.

This is useful to download large quantities of data from a THREDDS
server automatically breaking the large request in smaller requests
to avoid server timeout.

Parameters
----------
indexers : dict, optional
    A dict with keys matching dimensions and values given by positive
    integers.
    One of indexers or indexers_kwargs must be provided.
seconds_between_requests : float, optional
    Wait time in seconds between requests.
**indexers_kwargs : {dim: indexer, ...}, optional
    The keyword arguments form of ``indexers``.
    One of indexers or indexers_kwargs must be provided.

Returns
-------
xr.DataArray
   

For DataArrays, you can also load by bytesize instead of worrying about how many "steps".

In [37]:
da2 = da.lbs.load_by_bytesize(time="100KB")

Loading '90.1KB' of 'Tair' between time=[2014-12-23T12:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|███████████████████████████████| 79/79 [00:00<00:00, 127.95it/s]


Unlike `load_by_step`, at the moment `load_by_bytesize` only allows splitting along one dimension. This can be a problem, for example, if a single time step of your dataset is larger than the bytesize limit. This can happen when getting data from high resolution ocean models:
3600 (lons) x 1800 (lats) x 50 (depths) x 4 (bytes) = 1.2 GB

In [38]:
da2 = da.lbs.load_by_bytesize(time="1KB")

ValueError: It is not possible to load blocks of '1.0KB' along dimension 'time' even when using step=1. Consider increasing the size or calling split_by_step and splitting along a second dimension.

We are currenlty working on a new version that can split the request along multiple dimensions following a prefered order of dimensions, e.g.:

`da.lbs.load_by_bytesize(bytesize="50MB", dims=["time", "lat", "lon"])`



Besides the DataArray accessor, we also have a Dataset accessor. This will automatically iterate over all the DataArrays in the Dataset.

In [40]:
for var in list(ds.data_vars):
    print(var, ds[var]._in_memory)

Tair False
dTdx False
dTdy False


In [43]:
ds2 = ds.lbs.load_by_step(time=100)

Loading '53.0KB' of 'Tair' between time=[2014-12-27T00:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|████████████████████████████████| 30/30 [00:00<00:00, 99.36it/s]
Loading '106.0KB' of 'dTdx' between time=[2014-12-27T00:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|██████████████████████████████| 30/30 [00:00<00:00, 113.79it/s]
Loading '106.0KB' of 'dTdy' between time=[2014-12-27T00:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|██████████████████████████████| 30/30 [00:00<00:00, 157.09it/s]


With this, now we have the full Dataset in memory.

In [45]:
for var in list(ds.data_vars):
    print(var, ds[var]._in_memory, ds2[var]._in_memory)

Tair False True
dTdx False True
dTdy False True


This module attempts to solver the server access bottleneck problem. But at some point, the ammount of physical RAM memory available on the local computer will become a new problem.

As an example: most computers nowadays have at least 8 or 16GB of RAM. Assuming that the Operational System and other programs use an average of 4GB (assume way less for Linux and way more for Windows), this leaves 12GB of available memory. For a large Dataset with a lot of variables (DataArrays), even 12GB can not be enough. In these case we recommend the use of the `load_and_save_by_step` method.







In [47]:
ds.lbs.load_and_save_by_step(time=100, outfile="/tmp/foobar.nc")

Loading '53.0KB' of 'Tair' between time=[2014-12-27T00:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|███████████████████████████████| 30/30 [00:00<00:00, 100.59it/s]
/home/eani/ohw24_proj_xarray_load_by_step_us/load_by_step/_load_by_step.py:496: SerializationWarning: saving variable Tair with floating point data as an integer dtype without any _FillValue to use for NaNs
  da_in_memory.to_netcdf(outfile,
Loading '106.0KB' of 'dTdx' between time=[2014-12-27T00:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|██████████████████████████████| 30/30 [00:00<00:00, 196.88it/s]
Loading '106.0KB' of 'dTdy' between time=[2014-12-27T00:00:00.000000000, 2014-12-31T18:00:00.000000000]: 100%|██████████████████████████████| 30/30 [00:00<00:00, 126.81it/s]


This method loads each DataArray into memory, saves it to disk, and releases memory to load the next DataArray. With this the physical RAM memory available on the local computer only has to be capable of holding a single DataArray at a time, instead of the whole Dataset.

The previous examples made use of small local files for demostration purposes. Now we are going to show some real life examples accessing data from remote servers.